<a href="https://colab.research.google.com/github/Artmagic/GP2_team34/blob/main/%D0%94%D0%9E%D0%91%D0%90%D0%92%D0%9B%D0%95%D0%9D%D0%98%D0%95_%D0%A0%D0%95%D0%99%D0%A2%D0%98%D0%9D%D0%93%D0%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
import requests
import time
import random
import logging
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import os
from datetime import datetime

In [11]:
df = pd.read_csv('food_moscow_full.csv')

In [12]:
df.head()

,ID,Name,global_id,IsNetObject,OperatingCompany,TypeObject,AdmArea,District,Address,PublicPhone,SeatsCount,SocialPrivileges,Longitude_WGS84,Latitude_WGS84,geoData
0,77874,Родник,637376331,нет,NaN,кафе,Центральный административный округ,Таганский район,"город Москва, улица Талалихина, дом 2/1, корпус 1",(495) 676-55-35,35,нет,37.673306,55.738239,"{'coordinates': [37.67330613, 55.738238655], '..."
1,24309,Кафе «Академия»,637376349,нет,NaN,ресторан,Центральный административный округ,Таганский район,"Российская Федерация, город Москва, внутригоро...",(495) 662-30-10,95,нет,37.669648,55.735511,"{'coordinates': [37.669647597, 55.735511472], ..."
2,27429,ГБОУ «Школа № 1430 имени Героя Социалистическо...,637376480,нет,NaN,столовая,Северо-Восточный административный округ,район Лианозово,"Российская Федерация, город Москва, внутригоро...",(499) 908-06-15,240,нет,37.566940,55.904019,"{'coordinates': [37.56694, 55.904018801], 'typ..."
3,148815,Брусника,637376481,да,Брусника,кафе,Центральный административный округ,район Арбат,"Российская Федерация, город Москва, внутригоро...",(495) 697-04-89,10,нет,37.598089,55.747449,"{'coordinates': [37.598089, 55.747448755], 'ty..."
4,23394,Хинкальная Гогиели,637376678,нет,NaN,кафе,Юго-Восточный административный округ,район Лефортово,"Российская Федерация, город Москва, внутригоро...",(495) 361-38-50,24,нет,37.719950,55.749276,"{'coordinates': [37.719950379, 55.749275989], ..."


In [13]:
df['TypeObject'].unique()

array(['кафе', 'ресторан', 'столовая', 'буфет',
       'предприятие быстрого обслуживания', 'бар', 'кафетерий',
       'закусочная', 'ночной клуб (дискотека)',
       'магазин (отдел кулинарии)', 'заготовочный цех'], dtype=object)

In [14]:
df = df[df.TypeObject != "ночной клуб (дискотека)"]
df.head()

,ID,Name,global_id,IsNetObject,OperatingCompany,TypeObject,AdmArea,District,Address,PublicPhone,SeatsCount,SocialPrivileges,Longitude_WGS84,Latitude_WGS84,geoData
0,77874,Родник,637376331,нет,NaN,кафе,Центральный административный округ,Таганский район,"город Москва, улица Талалихина, дом 2/1, корпус 1",(495) 676-55-35,35,нет,37.673306,55.738239,"{'coordinates': [37.67330613, 55.738238655], '..."
1,24309,Кафе «Академия»,637376349,нет,NaN,ресторан,Центральный административный округ,Таганский район,"Российская Федерация, город Москва, внутригоро...",(495) 662-30-10,95,нет,37.669648,55.735511,"{'coordinates': [37.669647597, 55.735511472], ..."
2,27429,ГБОУ «Школа № 1430 имени Героя Социалистическо...,637376480,нет,NaN,столовая,Северо-Восточный административный округ,район Лианозово,"Российская Федерация, город Москва, внутригоро...",(499) 908-06-15,240,нет,37.566940,55.904019,"{'coordinates': [37.56694, 55.904018801], 'typ..."
3,148815,Брусника,637376481,да,Брусника,кафе,Центральный административный округ,район Арбат,"Российская Федерация, город Москва, внутригоро...",(495) 697-04-89,10,нет,37.598089,55.747449,"{'coordinates': [37.598089, 55.747448755], 'ty..."
4,23394,Хинкальная Гогиели,637376678,нет,NaN,кафе,Юго-Восточный административный округ,район Лефортово,"Российская Федерация, город Москва, внутригоро...",(495) 361-38-50,24,нет,37.719950,55.749276,"{'coordinates': [37.719950379, 55.749275989], ..."


In [15]:
df = df[~df['Name'].str.contains("школа", case=False, na=False)].reset_index(drop=True)
df

,ID,Name,global_id,IsNetObject,OperatingCompany,TypeObject,AdmArea,District,Address,PublicPhone,SeatsCount,SocialPrivileges,Longitude_WGS84,Latitude_WGS84,geoData
0,77874,Родник,637376331,нет,NaN,кафе,Центральный административный округ,Таганский район,"город Москва, улица Талалихина, дом 2/1, корпус 1",(495) 676-55-35,35,нет,37.673306,55.738239,"{'coordinates': [37.67330613, 55.738238655], '..."
1,24309,Кафе «Академия»,637376349,нет,NaN,ресторан,Центральный административный округ,Таганский район,"Российская Федерация, город Москва, внутригоро...",(495) 662-30-10,95,нет,37.669648,55.735511,"{'coordinates': [37.669647597, 55.735511472], ..."
2,148815,Брусника,637376481,да,Брусника,кафе,Центральный административный округ,район Арбат,"Российская Федерация, город Москва, внутригоро...",(495) 697-04-89,10,нет,37.598089,55.747449,"{'coordinates': [37.598089, 55.747448755], 'ty..."
3,23394,Хинкальная Гогиели,637376678,нет,NaN,кафе,Юго-Восточный административный округ,район Лефортово,"Российская Федерация, город Москва, внутригоро...",(495) 361-38-50,24,нет,37.719950,55.749276,"{'coordinates': [37.719950379, 55.749275989], ..."
4,22579,Алло Пицца,637376719,да,Алло Пицца,кафе,Западный административный округ,район Солнцево,"Российская Федерация, город Москва, внутригоро...",(495) 934-31-00,32,нет,37.398531,55.645103,"{'coordinates': [37.398531, 55.645102725], 'ty..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22095,571438,Масала Хаус индийская кухня,2824485162,нет,NaN,кафе,Юго-Восточный административный округ,район Выхино-Жулебино,"Российская Федерация, город Москва, внутригоро...",(977) 620-97-95,20,нет,37.851943,55.686787,"{'coordinates': [37.851943, 55.686787], 'type'..."
22096,571451,GREEN TERRA,2824485169,нет,NaN,предприятие быстрого обслуживания,Западный административный округ,район Кунцево,"Российская Федерация, город Москва, внутригоро...",(929) 500-00-76,1,нет,37.401644,55.743745,"{'coordinates': [37.401644, 55.743745], 'type'..."
22097,571463,NOBA кофейня бистро кондитерская,2824485176,нет,NaN,кафе,Южный административный округ,район Чертаново Южное,"Российская Федерация, город Москва, внутригоро...",(967) 183-36-06,15,нет,37.602714,55.589203,"{'coordinates': [37.602714, 55.589203], 'type'..."
22098,571482,Surf coffee,2824485182,да,Surf Coffee,кафетерий,Юго-Восточный административный округ,Южнопортовый район,"Российская Федерация, город Москва, внутригоро...",(495) 600-95-27,8,нет,37.690686,55.720139,"{'coordinates': [37.690686, 55.720139], 'type'..."


In [16]:
API_KEYS = [
    "501de12e-62e8-4183-a5ec-1dc18cb1d25c", "f2832f57-972e-4d27-a0a7-f085c1c3416c",
    "4c82e284-6425-41e6-ac2d-64513d4b26b5", "68e68d3e-0de5-432e-bfac-f0db603bb3ad",
    "a2e3fca2-c86b-4531-9ec5-e54396a215be", "6783839a-ddd5-4f09-a0c5-8d2813f6322d",
    "5bf9077a-24d0-409c-9865-f6cedf110ec2", "36fba40b-05ff-4ad7-9139-2e1980261e91",
    "c2e6194d-a2f6-4d66-aa58-a6b66c070656", "45960050-666b-4a16-89b6-d31c6498d6fb",
    "b1ad8df2-9596-4204-ad94-44aadf434d31", "d9205b22-231a-4006-bf8d-eadb7b8cee5b",
    "99a57644-7ffd-43a6-8ff7-160d5c7a1863", "91009ee5-cda7-4fdd-a122-58e75c16f87b",
    "672c4eca-f7cd-4c71-9467-e17fc7faede1", "b4220d99-6df3-4ca2-9b4f-88e37c982da0",
    "2a30a97d-ca71-4d9e-977e-3a792ebca0ac", "70c1c83a-ab1c-4269-b824-902e2518b1e9",
    "665df620-fef0-4f67-896d-8cadd1391b20", "557350fc-6eaa-4c6c-b9b6-46d4232a9e2e",
    "38185e53-2b46-4802-a7f3-2071444c0157", "0b8e743a-a44f-4e40-9e28-db9c91c7a8d5",
    "d0772be3-facd-479b-b81b-2ddda6ed5769"
]

OUTPUT_FILE = "food_with_ratings_full.csv"
LOG_FILE = "parser_log.txt"

SAVE_EVERY = 500
RADIUS = 300
KEY_LIMIT = 1000
MAX_WORKERS = len(API_KEYS)

# Настройка logging
os.makedirs("logs", exist_ok=True)
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
LOG_FILE = f"logs/parser_{timestamp}.log"

logger = logging.getLogger("parser")
logger.setLevel(logging.INFO)

if logger.hasHandlers():
    logger.handlers.clear()

file_handler = logging.FileHandler(LOG_FILE, encoding="utf-8")
file_handler.setFormatter(logging.Formatter("%(asctime)s [%(levelname)s] [%(threadName)s] %(message)s"))

console_handler = logging.StreamHandler()
console_handler.setFormatter(logging.Formatter("%(asctime)s | %(message)s"))

logger.addHandler(file_handler)
logger.addHandler(console_handler)

logger.info("Старт многопоточного парсинга с 23 API-ключами")


df["found_name"] = None
df["rating"] = None
df["reviews_count"] = None

# Разделяем DataFrame
split_size = len(df) // len(API_KEYS)
chunks = [df.iloc[i:i + split_size].copy() for i in range(0, len(df), split_size)]

for t_id, chunk in enumerate(chunks, start=1):
    logger.info(f"Ключ {t_id}: строки {chunk.index.min()} — {chunk.index.max()} (всего {len(chunk)})")

# Функция потока
def process_chunk(df_chunk, key, thread_id):
    local_results = []
    logger.info(f"Поток {thread_id} стартовал (ключ {key[:8]}..., {len(df_chunk)} записей)")

    for i, row in enumerate(df_chunk.itertuples()):
        params = {
            "q": row.Name,
            "location": f"{row.Longitude_WGS84},{row.Latitude_WGS84}",
            "radius": RADIUS,
            "page_size": 5,
            "fields": "items.name,items.address_name,items.reviews,items.point",
            "key": key
        }

        try:
            r = requests.get("https://catalog.api.2gis.com/3.0/items", params=params, timeout=8)
            if r.status_code != 200:
                logger.warning(f"Поток {thread_id}: ошибка {r.status_code} ({row.Name[:20]})")
                continue

            items = r.json().get("result", {}).get("items", [])
            if not items:
                continue

            item = items[0]
            reviews = item.get("reviews", {})
            rating = reviews.get("org_rating") or reviews.get("general_rating")
            review_count = reviews.get("org_review_count") or reviews.get("general_review_count")

            local_results.append((row.Index, item.get("name"), rating, review_count))

        except Exception as e:
            logger.error(f"Поток {thread_id}: ошибка при {row.Name[:20]} — {e}")
            continue

        # лог активности
        if (i + 1) % 100 == 0:
            logger.info(f"Поток {thread_id}: {i + 1}/{len(df_chunk)} строк обработано")

        # автосохранение
        if (i + 1) % SAVE_EVERY == 0:
            part_path = f"partial_thread_{thread_id}.csv"
            pd.DataFrame(local_results, columns=["idx", "found_name", "rating", "reviews_count"]).assign(thread=thread_id).to_csv(
                part_path, index=False, encoding="utf-8-sig"
            )
            logger.info(f"Поток {thread_id}: частичное сохранение ({i + 1} строк)")

        # ограничение по ключу
        if i + 1 >= KEY_LIMIT:
            logger.warning(f"Поток {thread_id}: достиг лимита 1000 запросов (ключ {key[:8]}...)")
            break

        # задержка
        time.sleep(random.uniform(0.25, 0.45))

    logger.info(f"Поток {thread_id} завершил ({len(local_results)} записей сохранено)")
    return local_results


# Запуск потоков
logger.info(f"Всего строк: {len(df)}, запуск {MAX_WORKERS} потоков")

results = []
with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    futures = []
    for t_id, (chunk, key) in enumerate(zip(chunks, API_KEYS), start=1):
        futures.append(executor.submit(process_chunk, chunk, key, t_id))

    for f in tqdm(as_completed(futures), total=len(futures), desc="Завершение потоков"):
        results.extend(f.result())

# Объединяем
for idx, name, rating, reviews in results:
    df.at[idx, "found_name"] = name
    df.at[idx, "rating"] = rating
    df.at[idx, "reviews_count"] = reviews

df.to_csv(OUTPUT_FILE, index=False, encoding="utf-8-sig")

for handler in logger.handlers:
    handler.flush()
    handler.close()

logger.info("Готово. Результаты сохранены в %s", OUTPUT_FILE)

2025-11-05 22:44:21,002 | Старт многопоточного парсинга с 25 API-ключами
INFO:parser:Старт многопоточного парсинга с 25 API-ключами
2025-11-05 22:44:21,023 | Ключ 1: строки 0 — 959 (всего 960)
INFO:parser:Ключ 1: строки 0 — 959 (всего 960)
2025-11-05 22:44:21,025 | Ключ 2: строки 960 — 1919 (всего 960)
INFO:parser:Ключ 2: строки 960 — 1919 (всего 960)
2025-11-05 22:44:21,028 | Ключ 3: строки 1920 — 2879 (всего 960)
INFO:parser:Ключ 3: строки 1920 — 2879 (всего 960)
2025-11-05 22:44:21,030 | Ключ 4: строки 2880 — 3839 (всего 960)
INFO:parser:Ключ 4: строки 2880 — 3839 (всего 960)
2025-11-05 22:44:21,033 | Ключ 5: строки 3840 — 4799 (всего 960)
INFO:parser:Ключ 5: строки 3840 — 4799 (всего 960)
2025-11-05 22:44:21,036 | Ключ 6: строки 4800 — 5759 (всего 960)
INFO:parser:Ключ 6: строки 4800 — 5759 (всего 960)
2025-11-05 22:44:21,038 | Ключ 7: строки 5760 — 6719 (всего 960)
INFO:parser:Ключ 7: строки 5760 — 6719 (всего 960)
2025-11-05 22:44:21,041 | Ключ 8: строки 6720 — 7679 (всего 960)
I

In [17]:
df2 = pd.read_csv('food_with_ratings_full.csv')
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22100 entries, 0 to 22099
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID                22100 non-null  int64  
 1   Name              22100 non-null  object 
 2   global_id         22100 non-null  int64  
 3   IsNetObject       22100 non-null  object 
 4   OperatingCompany  4115 non-null   object 
 5   TypeObject        22100 non-null  object 
 6   AdmArea           22100 non-null  object 
 7   District          22100 non-null  object 
 8   Address           22100 non-null  object 
 9   PublicPhone       22079 non-null  object 
 10  SeatsCount        22100 non-null  int64  
 11  SocialPrivileges  22100 non-null  object 
 12  Longitude_WGS84   22100 non-null  float64
 13  Latitude_WGS84    22100 non-null  float64
 14  geoData           22100 non-null  object 
 15  found_name        22011 non-null  object 
 16  rating            20116 non-null  float6